In [43]:
import pandas as pd
import plotly.express as px
import MetaTrader5 as mt5
from datetime import datetime
from IPython.display import display

from utils.mt5_utils import get_ohlc, get_ticks

In [ ]:
# how to retrieve data
# how to calculate price change and ranges
# which day is the most profitable?
# largest gain/drop
# longest up-day / down-day streak

In [44]:
mt5.initialize()


True

In [59]:
bars = get_ohlc('DE40',  mt5.TIMEFRAME_D1, datetime(2013, 1, 1), datetime.now())

cols = ['time', 'open', 'high', 'low', 'close']
bars = bars[cols]

bars['change'] = (bars['close'] - bars['open']) / bars['open']
bars['range'] = (bars['high'] - bars['low']) / bars['open']
bars['day'] = bars['time'].dt.dayofweek
bars['week'] = bars['time'].dt.isocalendar().week

bars

,time,open,high,low,close,change,range,day,week
0,2013-01-02,7732.8,7793.0,7731.5,7777.6,0.005794,0.007953,2,1
1,2013-01-03,7775.2,7777.5,7733.8,7745.8,-0.003781,0.005620,3,1
2,2013-01-04,7747.0,7747.6,7739.8,7741.6,-0.000697,0.001007,4,1
3,2013-01-07,7771.5,7774.5,7711.8,7727.3,-0.005687,0.008068,0,2
4,2013-01-08,7726.7,7754.8,7689.8,7705.2,-0.002783,0.008412,1,2
...,...,...,...,...,...,...,...,...,...
2665,2023-06-19,16342.9,16351.1,16186.3,16211.2,-0.008059,0.010084,0,25
2666,2023-06-20,16219.7,16219.7,16066.3,16125.7,-0.005795,0.009458,1,25
2667,2023-06-21,16131.1,16156.9,15993.8,15995.9,-0.008381,0.010111,2,25
2668,2023-06-22,16002.7,16008.1,15810.3,15992.3,-0.000650,0.012360,3,25


In [70]:
px.line(bars, x='time', y='change', hover_data='day')

In [60]:
px.line(bars, x='time', y='range', hover_data=['day'])

In [61]:
# volatility by day
vol_by_day = bars.groupby('day', as_index=False)['range'].mean()
px.bar(vol_by_day, x='day', y='range')

In [62]:
# volatility by day
change_by_day = bars.groupby('day', as_index=False)['change'].sum()
display(change_by_day)

px.bar(change_by_day, x='day', y='change', title='DAX Price sum(price_change) by day since 2022')

,day,change
0,0,0.486067
1,1,0.544054
2,2,0.432788
3,3,-0.229482
4,4,0.131771


In [63]:
px.line(bars, x='time', y='change', hover_data=['day'])

In [69]:
bars_by_week = bars.groupby('week', as_index=False).agg(
    daily_change=('change', list),
    prev_week=('change', 'sum')
)

bars_by_week

,week,daily_change,prev_week
0,1,"[0.0057935030002069345, -0.003781253215351327,...",0.013981
1,2,"[-0.005687447725664263, -0.0027825591779155395...",0.011209
2,3,"[0.0009456203528589059, -0.004425637641212757,...",0.153423
3,4,"[0.002928447404566352, -0.0014083779104322864,...",-0.019144
4,5,"[-0.003943518636305814, 0.0030393584144253662,...",-0.101038
5,6,"[-0.0255349045155521, 0.0030686109945446435, -...",-0.046130
6,7,"[-0.0018058099973829078, 0.004274288711157777,...",0.170118
7,8,"[0.00552289563177176, 0.01620218650866469, -0....",-0.011365
8,9,"[-0.007683507111469404, -0.0007468455601997782...",-0.108780
9,10,"[0.003854589817134082, 0.01461995863495351, 0....",0.085224
